In [1]:
import os
import sys
import pickle

import cv2
import numpy as np
import neuroglancer
from tqdm import tqdm
from cloudvolume import CloudVolume

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.sqlcontroller import SqlController
from utils import get_structure_number, get_segment_properties, NumpyToNeuroglancer

Connecting dklab@db.dk.ucsd.edu:3306


In [2]:
animal = 'MD589'
downsample_factor = 16

OUTPUT_DIR_PATH = os.path.join('./')
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full'
# NG_DIR_PATH = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/', animal, 'neuroglancer_data')
NG_DIR_PATH = os.path.join('./')

sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

In [3]:
def draw_numpy(section_structure_polygons, section_start, section_end, draw_lines=True):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    color = get_structure_number(structure)
                    
                    if draw_lines:
                        cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    else:
                        for point in polygon:
                            cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

def get_ranges(start, end, interval):
    ranges = []
    
    cur = start
    while cur + interval <= end:
        next = cur + interval
        ranges.append((cur, next))
        cur = next
    ranges.append((cur, end))
    return ranges

In [4]:
draw_lines = True
polygons_path = os.path.join(OUTPUT_DIR_PATH, f'MD589_CVAT_downsample{downsample_factor}.pkl')

with open(polygons_path, 'rb') as file:
    section_structure_polygons = pickle.load(file)
volume = draw_numpy(section_structure_polygons, 0, num_section, draw_lines=draw_lines)

100%|██████████| 447/447 [00:54<00:00,  8.22it/s]


In [5]:
precompute_path = os.path.join(NG_DIR_PATH, f'{animal}_down{downsample_factor}_annotations')

ng = NumpyToNeuroglancer(volume, scales)
ng.preview()
ng.init_precomputed(precompute_path)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[ 7232.  7232. 20000.] is added to:
http://127.0.0.1:38475/v/d30cdf03acf1282d835f0936383d3f41aa261aa6/


Tasks:   0%|          | 0/14 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[2731, 2025, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[2731, 2025, 447], dtype=int32)



Downloading:   0%|          | 0/1024 [00:00<?, ?it/s]


Downloading:   8%|▊         | 82/1024 [00:03<00:01, 538.05it/s]


...

Downloading:  81%|████████  | 362/448 [00:15<01:31,  1.06s/it]



Downloading:  67%|██████▋   | 300/448 [00:15<02:37,  1.06s/it]



Downloading:  67%|██████▋   | 301/448 [00:15<02:36,  1.06s/it]



Downloading:  67%|██████▋   | 302/448 [00:15<02:34,  1.06s/it]



Downloading:  68%|██████▊   | 303/448 [00:15<02:33,  1.06s/it]



Downloading:  68%|██████▊   | 304/448 [00:15<02:32,  1.06s/it]



Downloading:  81%|████████  | 363/448 [00:15<01:30,  1.06s/it]



Downloading:  81%|████████▏ | 364/448 [00:15<01:28,  1.06s/it]



Downloading:  68%|██████▊   | 305/448 [00:15<02:30,  1.05s/it]



Downloading:  68%|██████▊   | 306/448 [00:15<02:29,  1.05s/it]



Downloading:  69%|██████▊   | 307/448 [00:15<02:27,  1.05s/it]



Downloading:  69%|██████▉   | 308/448 [00:15<02:26,  1.05s/it]



Downloading:  69%|██████▉   | 309/448 [00:15<02:25,  1.05s/it]



Downloading:  69%|██████▉   | 310/448 [00:15<02:24,  1.04s/it]



Downloading:  81%|████████▏ | 365/448 [00:15<01:27,  1.05s/it]



Downloadin

Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]






Downloading:   0%|          | 0/224 [00:00<?, ?it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading: 100%|██████████| 448/448 [00:17<00:00,  1.01it/s]



Downloading:  88%|████████▊ | 395/448 [00:17<00:50,  1.05it/s]



Downloading:  88%|████████▊ | 396/448 [00:17<00:51,  1.01it/s]



Downloading:  89%|████████▊ | 397/448 [00:17<00:52,  1.02s/it]



Downloading:  89%|████████▉ | 398/448 [00:17<00:52,  1.04s/it]



Downloading:  89%|████████▉ | 399/448 [00:17<00:52,  1.06s/it]



Downloading:  89%|████████▉ | 400/448 [00:17<00:51,  1.07s/it]



Downloading:  90%

Downloading:  85%|████████▍ | 380/448 [00:19<01:03,  1.07it/s]



Downloading:  85%|████████▌ | 381/448 [00:19<01:00,  1.11it/s]



Downloading:  85%|████████▌ | 382/448 [00:19<00:57,  1.15it/s]



Downloading:  85%|████████▌ | 383/448 [00:19<00:55,  1.17it/s]



Downloading:  86%|████████▌ | 384/448 [00:19<00:53,  1.19it/s]



Downloading:  98%|█████████▊| 439/448 [00:19<00:08,  1.11it/s]



Downloading:  98%|█████████▊| 440/448 [00:19<00:07,  1.14it/s]



Downloading:  86%|████████▌ | 385/448 [00:19<00:52,  1.20it/s]



Downloading:  86%|████████▌ | 386/448 [00:19<00:51,  1.21it/s]



Downloading:  86%|████████▋ | 387/448 [00:19<00:50,  1.21it/s]



Downloading:  87%|████████▋ | 388/448 [00:19<00:49,  1.22it/s]



Downloading:  87%|████████▋ | 389/448 [00:19<00:48,  1.22it/s]



Downloading:  87%|████████▋ | 390/448 [00:19<00:47,  1.23it/s]



Downloading:  98%|█████████▊| 441/448 [00:19<00:06,  1.16it/s]



Downloading:  87%|████████▋ | 391/448 [00:19<00:46,  1.23it/s]



Downloadin

Downloading:  94%|█████████▍| 421/448 [00:21<00:30,  1.14s/it]



Downloading:  94%|█████████▍| 422/448 [00:21<00:27,  1.07s/it]



Downloading:  94%|█████████▍| 423/448 [00:21<00:25,  1.03s/it]



Downloading:  95%|█████████▍| 424/448 [00:21<00:23,  1.00it/s]



Downloading:  95%|█████████▍| 425/448 [00:21<00:22,  1.02it/s]



Downloading:  95%|█████████▌| 426/448 [00:21<00:20,  1.05it/s]



Downloading:  95%|█████████▌| 427/448 [00:21<00:19,  1.07it/s]



Downloading:  96%|█████████▌| 428/448 [00:21<00:18,  1.08it/s]



Downloading:  96%|█████████▌| 429/448 [00:21<00:17,  1.09it/s]



Downloading:  96%|█████████▌| 430/448 [00:21<00:16,  1.10it/s]



Downloading:  96%|█████████▌| 431/448 [00:21<00:15,  1.10it/s]



Downloading:  96%|█████████▋| 432/448 [00:21<00:14,  1.11it/s]



Downloading:  97%|█████████▋| 433/448 [00:21<00:13,  1.11it/s]



Downloading:  97%|█████████▋| 434/448 [00:21<00:12,  1.12it/s]



Downloading:  97%|█████████▋| 435/448 [00:21<00:11,  1.12it/s]



Downloadin

Downloading:  89%|████████▉ | 199/224 [00:06<00:04,  5.24it/s]






Downloading:  89%|████████▉ | 200/224 [00:06<00:06,  3.96it/s]






Downloading:  90%|████████▉ | 201/224 [00:06<00:06,  3.38it/s]






Downloading:  90%|█████████ | 202/224 [00:06<00:07,  3.05it/s]






Downloading:  91%|█████████ | 203/224 [00:06<00:07,  2.86it/s]






Downloading:  91%|█████████ | 204/224 [00:06<00:07,  2.74it/s]






Downloading:  92%|█████████▏| 205/224 [00:06<00:07,  2.66it/s]






Downloading:  92%|█████████▏| 206/224 [00:06<00:06,  2.61it/s]






Downloading:  92%|█████████▏| 207/224 [00:06<00:06,  2.58it/s]






Downloading:  93%|█████████▎| 208/224 [00:06<00:06,  2.56it/s]






Downloading:  93%|█████████▎| 209/224 [00:06<00:05,  2.55it/s]






Downloading:  94%|█████████▍| 210/224 [00:06<00:05,  2.53it/s]






Downloading:  94%|█████████▍| 211/224 [00:06<00:05,  2.53it/s]






Downloading:  95%|█████████▍| 212/224 [00:06<00:04,  2.53it/s]






Downloading:  95%|█████████▌| 213/